In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import time
import os
import gdreg
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from statsmodels.stats.multitest import multipletests


# autoreload
%load_ext autoreload
%autoreload 2

### Trait info

In [2]:
DIC_NAME = {
    'blood_PLATELET_COUNT': 'Platelet Count', 
    'blood_RBC_DISTRIB_WIDTH': 'Red Blood Cell Distribution Width',
    'blood_RED_COUNT': 'Red Blood Cell Count',
    'blood_WHITE_COUNT': 'White Blood Cell Count', 
    'bmd_HEEL_TSCOREz': 'Heel T Score', 
    'body_BALDING1': 'Balding Type I', 
    'body_BMIz': 'BMI',
    'body_HEIGHTz': 'Height',
    'body_WHRadjBMIz': 'Waist-hip Ratio',
    'bp_DIASTOLICadjMEDz': 'Diastolic Blood Pressure',
    'cov_EDU_YEARS': 'Years of Education',
    'disease_ALLERGY_ECZEMA_DIAGNOSED': 'Eczema',
    'lung_FEV1FVCzSMOKE': 'FEV1-FVC Ratio',
    'lung_FVCzSMOKE': 'Forced Vital Capacity (FVC)',
    'mental_NEUROTICISM': 'Neuroticism',
    'other_MORNINGPERSON': 'Morning Person',
    'pigment_SUNBURN': 'Sunburn Occasion',
    'repro_MENARCHE_AGE': 'Age at Menarche',
    'repro_MENOPAUSE_AGE': 'Age at Menopause',
    'repro_NumberChildrenEverBorn_Pooled': 'Number Children (Pooled)',
    'cancer_BREAST' : 'Breast Cancer',
    'cancer_PROSTATE' : 'Prostate Cancer',
    'disease_HYPOTHYROIDISM_SELF_REP' : 'Hypothyroidism',
    'biochemistry_AlkalinePhosphatase' : 'Alkaline Phosphatase',
    'biochemistry_AspartateAminotransferase' : 'Aspartate Aminotransferase',
    'biochemistry_Cholesterol' : 'Cholesterol', 
    'biochemistry_Creatinine' : 'Creatinine',
    'biochemistry_IGF1' : 'IGF1',
    'biochemistry_Phosphate' : 'Phosphate',
    'biochemistry_TotalBilirubin' : 'Total Bilirubin',
    'biochemistry_TotalProtein' : 'Total Protein',
    'biochemistry_VitaminD' : 'Vitamin D',
}

In [3]:
df_info = pd.DataFrame(index=list(DIC_NAME))
df_info['Trait Name'] = [DIC_NAME[x] for x in df_info.index]
df_info['Trait_Identifier'] = df_info.index

sumstats_file = '/n/groups/price/martin/data_GDREG/UKBimp_337K_MAF001/UKBB_trait/@.sumstats.gz'
df_info['N.ukbimp'] = [pd.read_csv(sumstats_file.replace('@', x), nrows=10, sep='\t')['N'].median()
                for x in df_info.index]
df_info['N.ukbimp'] = df_info['N.ukbimp'].astype(int)

df_info.to_csv('/n/groups/price/martin/data_GDREG/results/tables/trait_info.tsv', sep='\t', index=False)

### Single-SNP annotation info

In [2]:
# [Name, type, reference]
DIC_NAME_AN = {
    'BivFlnk' : ['Flanking bivalent TSS/enhancer', 'b', 'Roadmap Consortium 2015'],
    'Conserved_LindbladToh' : ['Conserved (Lindblad-Toh)', 'b', 'Ward and Kellis 2012'],
    'Conserved_Vertebrate_phastCons46way' : ['Conserved in vertebrates (phastCons)', 'b', 'Siepel et al. 2005'],
    'Conserved_Mammal_phastCons46way' : ['Conserved in mammals (phastCons)', 'b', 'Siepel et al. 2005'],
    'Conserved_Primate_phastCons46way' : ['Conserved in primates (phastCons)', 'b', 'Siepel et al. 2005'],
    'Coding_UCSC' : ['Coding', 'b', 'UCSC'],
    'UTR_3_UCSC' : ['3UTR', 'b', 'UCSC'],
    'UTR_5_UCSC' : ['5UTR', 'b', 'UCSC'],
    'Intron_UCSC' : ['Intron', 'b', 'UCSC'],
    'Enhancer_Andersson' : ['FANTOM5 enhancer', 'b', 'Andersson et al. 2014'],
    'SuperEnhancer_Hnisz' : ['Super enhancer (Hnisz)', 'b', 'Hnisz et al. 2013'],
    'Enhancer_Hoffman' : ['Enhancer', 'b', 'Hoffman et al. 2013'],
    'WeakEnhancer_Hoffman' : ['Weak enhancer', 'b', 'Hoffman et al. 2013'],
    'Promoter_UCSC' : ['Promoter', 'b', 'UCSC'],
    'PromoterFlanking_Hoffman' : ['Promoter flanking', 'b', 'Hoffman et al. 2013'],
    'DHS_Trynka' : ['DHS', 'b', 'Trynka et al. 2013'],
    'FetalDHS_Trynka' : ['Fetal DHS', 'b', 'Trynka et al. 2013'],
    'H3K27ac_Hnisz' : ['H3K27ac (Hnisz)', 'b', 'Hnisz et al. 2013'],
    'H3K27ac_PGC2' : ['H3K27ac (PGC2)', 'b', 'PGC 2014'],
    'H3K4me1_Trynka' : ['H3K4me1', 'b', 'Trynka et al. 2013'],
    'H3K4me3_Trynka' : ['H3K4me3', 'b', 'Trynka et al. 2013'],
    'H3K9ac_Trynka' : ['H3K9ac', 'b', 'Trynka et al. 2013'],
    'TFBS_ENCODE' : ['TFBS', 'b', 'ENCODE'],
    'Transcribed_Hoffman' : ['Transcribed', 'b', 'Hoffman et al. 2013'],
    'TSS_Hoffman' : ['TSS', 'b', 'Hoffman et al. 2013'],
    'CTCF_Hoffman' : ['CTCF', 'b', 'Hoffman et al. 2013'],
    'DGF_ENCODE' : ['DGF', 'b', 'ENCODE'],
    'Repressed_Hoffman' : ['Repressed', 'b', 'Hoffman et al. 2013'],
    'Vahedi_Tcell_SE' : ['Super enhancer (Vahedi)', 'b', 'Vahedi et al. 2015'],
    'Vahedi_Tcell_TE' : ['Typical enhancer', 'b', 'Vahedi et al. 2015'],
    'GerpRS_g4' : ['Conserved (GERP RS >= 4)', 'b', 'Davydov et al. 2010'], # from CADD
    'GerpN' : ['Conserved (GERP NS)', 'c', 'Davydov et al. 2010'], # from CADD
    'alleleage' : ['MAF-adjusted predicted allele age', 'c', 'Rasmussen et al. 2014'],
    'LLD_AFR' : ['MAF-adjusted LLD-AFR', 'c', 'Gazal et al. 2017'],
    'nucleotide_div' : ['Nucleotide diversity', 'c', ''],
    'recomb_rate' : ['Recombination rate', 'c', 'HapMap2 map'],
    'Backgrd_Selection_Stat' : ['McVicker B statistics', 'c', 'McVicker et al. 2009'],
    'CpG' : ['CpG content', 'c', ''], # from CADD
    # Non-synonymous removed
    # Synonymous removed
    'CADD_g20' : ['Deleterious (CADD >= 20)', 'b', 'Kircher et al. 2014'],
    'snpeff_nonsynonymous_variant' : ['Non-synonymous', 'b', 'Cingolani et al. 2012'],
    'snpeff_synonymous_variant' : ['Synonymous', 'b', 'Cingolani et al. 2012'],
}

In [23]:
def get_type(line):
    if line == 'b':
        return 'Binary'
    if line == 'c':
        return 'Continuous'

df_info = pd.DataFrame(index=DIC_NAME_AN)
df_info['Name'] = [DIC_NAME_AN[x][0] for x in df_info.index]
df_info['Identifier'] = df_info.index
df_info['Type'] = [get_type(DIC_NAME_AN[x][1]) for x in df_info.index]
df_info['n_snp_common'] = 0
df_info['n_snp_lf'] = 0
df_info['Reference/dataset'] = [DIC_NAME_AN[x][2] for x in df_info.index]

# n_snp
temp_df = pd.read_csv(
    '/n/groups/price/martin/data_GDREG/UKBimp_337K_MAF001/gdreg_res.baseline_100/blood_PLATELET_COUNT.tau.tsv',
    sep='\t', index_col=0
)
for row in df_info.index:
    if df_info.loc[row, 'Type'] == 'Binary':
        df_info.loc[row, 'n_snp_common'] = temp_df.loc['AN:%s_common'%row, 'n_snp']
        df_info.loc[row, 'n_snp_lf'] = temp_df.loc['AN:%s_lf'%row, 'n_snp']
df_info['n_snp_common'] = df_info['n_snp_common'].astype(int)
df_info['n_snp_lf'] = df_info['n_snp_lf'].astype(int)

# Version of the baseline model
df_ldsc = pd.read_excel('/n/groups/price/martin/data_GDREG/results/Gazal_NG_2018_table.xlsx',
                        sheet_name='S1', skiprows=1)
df_ldsc.index = df_ldsc['Main annotation']
df_ldsc.loc[df_ldsc['Version of the baseline model']=='New annotation', 'Version of the baseline model'] = \
    'Gazal et al. 2018' 
df_info['Version of the baseline model'] = [
    df_ldsc.loc[x, 'Version of the baseline model'] 
    if (x in df_ldsc.index) & (x not in ['Non-synonymous', 'Synonymous']) 
    else 'New annotation' 
    for x in df_info['Name']
]

df_info.to_csv('/n/groups/price/martin/data_GDREG/results/tables/annot_info.tsv', sep='\t', index=False)

### SNP-pair annotation info

In [24]:
DIC_NAME_pAN = {
    'proxy_0_100' : ['Proximal (0-0.1 kb)', 'All pairs of SNPs within 100 bp'],
    'proxy_0_1000' : ['Proximal (0-1 kb)', 'All pairs of SNPs within 1 kb'],
    'proxy_0_10000' : ['Proximal (0-10 kb)', 'All pairs of SNPs within 10 kb'],
    'proxy_100_1000' : ['Proximal (0.1-1 kb)', 'All pairs of SNPs between 100 bp and 1 kb'],
    'proxy_1000_10000' : ['Proximal (1-10 kb)', 'All pairs of SNPs between 1 kb and 10 kb'],
    'ldp5_proxy_10000' : ['High-LD', 'All pairs of SNPs with LD >= 0.5 and dist < 10 kb'],
    'ldp5_proxy_0_100' : ['High-LD (0-0.1 kb)', 'All pairs of SNPs with LD >= 0.5 and dist < 100 bp'],
    'ldp5_proxy_100_1000' : ['High-LD (0.1-1 kb)', 
                             'All pairs of SNPs with LD >= 0.5 and dist between 100 bp and 1 kb'],
    'ldp5_proxy_1000_10000' : ['High-LD (1-10 kb)', 
                                      'All pairs of SNPs with LD >= 0.5 and dist between 1 kb and 10 kb'],
    'exon' : ['Exon', 'All pairs of SNPs in the same exon'],
    'gene' : ['Gene', 'All pairs of SNPs in the same gene'],
    'exonic_gene' : ['Exonic-gene', 'All pair of exonic SNPs in the same gene body'],
    'protein_domain' : ['Protein-domain', 'All pair of SNPs in the same protein domain and same gene'],
    'cS2G_all' : ['cS2G-all', 'All pairs of SNPs linked to the same gene by cS2G'],
#     'cS2G_promoter' : ['cS2G-promoter', 'All pairs of promoter SNPs linked to the same gene by cS2G'],
    'cS2G_promoter' : ['Promoter', 'All pairs of promoter SNPs linked to the same gene by cS2G'],
    'cS2G_other' : [
        'cS2G-other', 
        'All pairs of other SNPs linked to the same gene by cS2G,  excluding exonic or promoter SNPs'],
    'Dixon_2015_H1_ESC' : ['Loop-ESC', 'Chromatin loops in H1 human embryonic stem (ES) cells'],
    'Dixon_2015_H1_MES' : ['Loop-MES', 'Chromatin loops in H1 human mesendoderm (ME) cell'],
    'Dixon_2015_H1_MSC' : ['Loop-MSC', 'Chromatin loops in H1 human mesenchymal stem (MS) cell'],
    'Dixon_2015_H1_NPC' : ['Loop-NPC', 'Chromatin loops in H1 human neural progenitor (NP) cell'],
    'Dixon_2015_H1_TRO' : ['Loop-TRO', 'Chromatin loops in H1 human trophoblast-like (TB) cell'],
}

In [61]:
df_info = pd.DataFrame(index=DIC_NAME_pAN)
df_info['Name'] = [DIC_NAME_pAN[x][0] for x in df_info.index]
df_info['Identifier'] = df_info.index
df_info['n_pair_common_common'] = 0
df_info['n_pair_common_lf'] = 0
df_info['n_pair_lf_lf'] = 0
df_info['Description'] = [DIC_NAME_pAN[x][1] for x in df_info.index]

# n_pair
STATS_FILE = '/n/groups/price/martin/data_GDREG/UKBimp_337K_MAF001/pannot/stats/@.dist.tsv'
for row in df_info.index:
    for mbin in ['common_common', 'common_lf', 'lf_lf']:
        if os.path.exists(STATS_FILE.replace('@', '%s_%s' % (row,mbin))):
            temp_df = pd.read_csv(STATS_FILE.replace('@', '%s_%s' % (row,mbin)), sep='\t')
            df_info.loc[row, 'n_pair_%s' % mbin] = temp_df['count'].sum()
            
df_info.to_csv('/n/groups/price/martin/data_GDREG/results/tables/pannot_info.tsv', sep='\t', index=False)